# Data Project - Gender Disparities in Employment and Unemployment Across Danish Industries
---

Gross unemployment rate is an important economic indicator that measures the percentage of the labor force that is unemployed and actively seeking work. In this project, we will focus on analyzing the gross unemployment rate in Denmark across different industries. By doing so, we aim to identify patterns and trends in the labor market that can help inform policy decisions and support efforts to reduce unemployment in the country. Specifically, we will explore how the gender gap in unemployment rates varies across different industries, and whether certain industries are more prone to higher levels of unemployment than others. To accomplish this, we will use data from Statistics Denmark and visualize our findings using interactive plots.

The project is presented as a "story" that aims to provide a more reader-friendly flow. Instead of importing all the necessary data at once and explaining it in detail, we visualize the area of interest and import the relevant data as we progress through the analysis.

This approach allows readers to follow along more easily and better understand how the analysis was conducted.

---
## Necesities

The imported modules in the code below will allow for general calculations, using the Statistics Denmark API to retrieve data, and creating interactive plots. The numpy and pandas libraries are used for data manipulation and analysis, while the dstapi module is used to retrieve data from the Statistics Denmark API. The matplotlib and plotly libraries are used to create visualizations of the data.

Furthermore for consistency sake, all figures are interactive. To select/deselect parts of the graphs simply use the legends.

>NOTE: This project uses DstAPI, which requires further installtion. See ReadMe for installtion guide

In [1]:
# For general Calculations and libraries
import numpy as np
import pandas as pd

# For using Statistics Denmark API
from dstapi import DstApi

# For interactive plots
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

---
## Gender disparity in gross unemployment (2007-2022)

To start with, we want to find data on gross unemployment in Denmark for the period 2007-2022. For this, data from table AULP01 is being used, which comes from the DST API and shows the number of "Full-time unemployed persons in per cent of the labor force by region, age and sex". The data is a register-based unemployment statistic, where a monthly total count is made of the gross unemployed registered in the unemployment funds and the public job centers in Denmark but is calculated in this table as the annual unemployment rate. A table summary is drawn up from this data, which can be seen below. After this, the parameters are defined where it is chosen to examine the gross unemployment for the whole of Denmark and for all ages, but with a division within gender, as it is this parameter that we want to examine the relationship on. This therefore means that with the "drop" function we remove the data division for area and age, as the focus is instead on gender, where the other parameters are therefore not investigated in this project. Data is sorted by time, where the unemployment percentages are also converted from being strings to floats. This can be displayed as an unemployment table, which is also seen below. In addition to this table, a graphical illustration of the gross unemployment rates for each men and women as well as the total rate is made as an interactive plot, which is seen as Figure 1 below.

In [2]:
# Searching for gross unemployment data through DST API
gross_unemployment_dst = DstApi('AULP01')

# Creating and displaying the a summary of the gross unemployment data
summary_gross_unemployment = gross_unemployment_dst.tablesummary(language='en')
summary_gross_unemployment

Table AULP01: Full-time unemployed persons in per cent of the labour force by region, age, sex and time
Last update: 2023-04-14T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,117,000,All Denmark,997,Abroad,False
1,ALDER,10,TOT,"Age, total",6099,60 year and over,False
2,KØN,3,TOT,Total,K,Women,False
3,Tid,16,2007,2007,2022,2022,True


In [3]:
# Setting parameters
par_gross_unemployment = {
    'table': 'AULP01',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'OMRÅDE', 'values': ['000']}, #000 = All of Denmark
        {'code': 'ALDER', 'values': ['TOT']}, #TOT = Age, total
        {'code': 'KØN', 'values': ['TOT', 'M', 'K']}, #TOT = Total, M = Men, K = Woman
        {'code': 'Tid', 'values': ['2006<=2022']}
    ]
}

# Fetching data based parameters set (Fetching from AUS07)
gross_unemployment = gross_unemployment_dst.get_data(params=par_gross_unemployment)

# Removing the 'OMRÅDE' and 'ALDER' columns
gross_unemployment = gross_unemployment.drop(['OMRÅDE', 'ALDER'], axis=1)

# Sort the DataFrame by date
gross_unemployment = gross_unemployment.sort_values('TID')

# Renaming the columns
gross_unemployment.rename(columns=
                          {'KØN':'GENDER',
                          'TID':'YEAR',
                          'INDHOLD':'GROSS UNEMPLOYMENT RATE (%)'},
                          inplace=True
                    )

# Convert pct. into floats
gross_unemployment['GROSS UNEMPLOYMENT RATE (%)'] = pd.to_numeric(gross_unemployment['GROSS UNEMPLOYMENT RATE (%)'])

# Displaying gross unemployment table
gross_unemployment.head(6)

,GENDER,YEAR,GROSS UNEMPLOYMENT RATE (%)
0,Women,2007,4.4
1,Men,2007,2.9
2,Total,2007,3.6
3,Women,2008,2.9
4,Men,2008,2.4
5,Total,2008,2.6


In [4]:
# Define colors for each line
colors = {'Total': 'green', 'Men': 'blue', 'Women': 'red'}

# Group the data by the 'TOTAL GROSS EMPLOYMENT AND SEX' column
grouped = gross_unemployment.groupby('GENDER')

# Create a figure object
fig_1 = px.line(title='Figure 1: Gross Unemployment as a pct. of the labor force by gender in Denmark')

# Loop through the grouped data and add each group to the figure with the defined color
for name, group in grouped:
    fig_1.add_scatter(x=group['YEAR'], y=group['GROSS UNEMPLOYMENT RATE (%)'], mode='lines', name=name, line=dict(color=colors[name]))

# Update the layout of the figure
fig_1.update_layout(xaxis_title='', yaxis_title='Gross Unemployment Rate (%)', title_font=dict(size=20), 
                  title_x=0.5, margin=dict(l=50, r=50, b=50, t=50, pad=4), hovermode='x unified',
                  xaxis=dict(tickmode='linear', dtick=1),
                  height = 500, width = 1000)

# Add a text box for the source
fig_1.add_annotation(text='Source: Statistics Denmark AULP01', xref='paper', yref='paper', x=0, y=0, showarrow=False, font=dict(size=10))

# Show the interactive plot
fig_1.show()

We look at Figure 1 men's and women's gross unemployment rate in percentage, where gross unemployment in 2007 and 2008 is seen to be decreasing, where in 2009 and 2010 it rises sharply, but worst for men. However, unemployment gradually falls in the years after the financial crisis as there is more confidence in the economy and activity increases. This increase slows down in 2020, which is primarily due to the corona epidemic, where unemployment is seen to rise until 2021, when it falls again and is seen to be at its lowest since 2008. 

With a new figure, we now want to illustrate the differences more clearly in gross unemployment between men and women in Denmark. We also call this the Gender Unemployment Gap and is found as the difference between the gross unemployment rates for women and men. In addition, a horizontal line is inserted at a Gender Unemployment Gap of zero to more clearly show when the trends reverse during the period. This is drawn as an interactive plot and can be seen below as Figure 2.

In [5]:
# Calculating difference between male and female gross unemployment rates
male_unemployment = gross_unemployment[gross_unemployment['GENDER'] == 'Men']
female_unemployment = gross_unemployment[gross_unemployment['GENDER'] == 'Women']

gender_gap = female_unemployment[['YEAR']].copy()
gender_gap['GENDER UNEMPLOYMENT GAP (%)'] = female_unemployment['GROSS UNEMPLOYMENT RATE (%)'].values - male_unemployment['GROSS UNEMPLOYMENT RATE (%)'].values

# Creating the plotly figure object
fig_2 = px.line(gender_gap, x='YEAR', y='GENDER UNEMPLOYMENT GAP (%)', title='Figure 2: Gender Unemployment Gap in Denmark (female unemployment - male unemployment)')

# Updating the layout
fig_2.update_layout(xaxis_title='', yaxis_title='Gender Unemployment Gap (%)', title_font=dict(size=20), 
                  title_x=0.5, margin=dict(l=50, r=50, b=50, t=50, pad=4), hovermode='x unified',
                  xaxis=dict(tickmode='linear', tick0=2007, dtick=1),
                  height = 500, width = 1000)

# Adding horizontal line for y=0
fig_2.add_shape(type='line', x0=2007, y0=0, x1=2022, y1=0, line=dict(color='black', width=0.5))

# Adding text box for source
fig_2.add_annotation(text='Source: Statistics Denmark AULP01', xref='paper', yref='paper', x=0, y=0, showarrow=False, font=dict(size=10))

# Show the interactive plot
fig_2.show()

In Figure 2, we can easily see the differences between the development of gross unemployment between men and women. A positive Gender Unemployment Gap refers to higher unemployment among women, where a negative gap means that men's unemployment is higher than women's. Unemployment in 2007 and 2008 is seen to be greatest among women, where the difference was respectively 1.5% and 0.5% compared to the men in the same period. In 2009, a shift in this development can be seen, where unemployment was significantly higher among men, which it can be seen to be until 2012. Here, the difference in inequality was between 0.5-1.8% in this period and indicates that the industries where most men are employed have been hardest hit by the financial crisis. In 2013, women's unemployment is again seen to be higher than men's, but where we also see a smaller difference between them, and where they follow each other's trends in development to a much greater extent, as the Gender Unemployment Gap is relatively stable in the following years.

---
## Gender disparity analysis for Danish industries (2008-2022)

Based on this inequality in unemployment between men and women, we want to find a correlation between the development of unemployment and real GDP, as we have a hypothesis that there will be a correlation. For this, data from table NAN1 is being used, which comes from the DST API and shows "Demand and supply by transaction and price unit". Here the column called Period-to-period real growth (pct.) is searched for and selected, as this is the one, we want to use. A table is made which shows a summary of the GDP data, where the parameters included are then defined. A fetching based on parameter seen from NAN1 is performed and the 'TRANSACT' and 'PRICE UNIT' columns are removed. Data is sorted by time, where the unemployment percentages are also converted from being strings to floats. This is shown as a table where you can see the percentage developments from year to year. However, a more visual illustration of this is desired. Therefore, a bar trade is made for GDP growth and a line trace for the gender unemployment gap. Next, we want to divide periods with vertical lines, which illustrate cyclical changes. Cyclical changes are defined here as a recession of less than 1.5% and economic climate is higher than 1.5%. To see if there should be a correlation between Real GDP Growth and Gender Unemployment Gap, both data are plotted on the same graph, which can be seen as Figure 3.

>NOTE Missing data for 2007

In [6]:
# Searching for GDP data through DST API
GDP_dst = DstApi('NAN1')

# Creating and displaying the a summary of the GDP data
summary_GDP_dst = GDP_dst.tablesummary(language='en')
summary_GDP_dst

Table NAN1: Demand and supply by transaction, price unit and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,TRANSAKT,31,B1GQK,B.1*g Gross domestic product,EMPM_DC,"Total employment (1,000 persons)",False
1,PRISENHED,6,V_M,"Current prices, (bill. DKK.)",LAN_C,"Pr. capita, 2010-prices, chained values, (1000...",False
2,Tid,57,1966,1966,2022,2022,True


In [7]:
# Setting parameters
par_GDP = {
    'table': 'NAN1',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'TRANSAKT', 'values': ['B1GQK']},
        {'code': 'PRISENHED', 'values': ['L_V']}, #L_V = Period-to-period real growth (pct.)
        {'code': 'Tid', 'values': ['>2006<=2022']}
    ]
}

# Fetching data based parameters set (Fetching from NAN1)
GDP = GDP_dst.get_data(params=par_GDP)

# Removing the 'TRANSAKT' and 'PRISENHED' columns
GDP = GDP.drop(['TRANSAKT', 'PRISENHED'], axis=1)

# Renaming the columns
GDP.rename(columns=
                {'TID':'YEAR',
                'INDHOLD':'REAL GROWTH IN GDP (%)'},
                inplace=True
            )

# Convert pct. into floats
GDP['REAL GROWTH IN GDP (%)'] = pd.to_numeric(GDP['REAL GROWTH IN GDP (%)'])

# Sort the DataFrame by date
GDP = GDP.sort_values('YEAR')

# Displaying GDP tabel
GDP.head(5)

,YEAR,REAL GROWTH IN GDP (%)
0,2007,0.9
1,2008,-0.5
2,2009,-4.9
3,2010,1.9
4,2011,1.3


In [8]:
# Create a bar trace for GDP growth
gdp_trace = go.Bar(x=GDP['YEAR'], y=GDP['REAL GROWTH IN GDP (%)'], name='Real GDP Growth')

# Create a line trace for the gender unemployment gap
gap_trace = go.Scatter(x=gender_gap['YEAR'], y=gender_gap['GENDER UNEMPLOYMENT GAP (%)'], mode='lines', name='Gender Unemployment Gap', line=dict(color='red'))

# Create a list of shapes for the vertical lines
shapes = []
for i in range(1, len(GDP)):
    if GDP['REAL GROWTH IN GDP (%)'][i] > 1.5 and GDP['REAL GROWTH IN GDP (%)'][i-1] <= 1.5:
        shapes.append(dict(type='line', xref='x', yref='paper', x0=GDP['YEAR'][i-1] + 0.5, y0=0, x1=GDP['YEAR'][i-1] + 0.5, y1=1, line=dict(color='magenta')))
    elif GDP['REAL GROWTH IN GDP (%)'][i] <= 1.5 and GDP['REAL GROWTH IN GDP (%)'][i-1] > 1.5:
        shapes.append(dict(type='line', xref='x', yref='paper', x0=GDP['YEAR'][i-1] + 0.5, y0=0, x1=GDP['YEAR'][i-1] + 0.5, y1=1, line=dict(color='magenta')))

# Define the layout
layout = go.Layout(
    title=dict(text='Figure 3: Real GDP Growth and Gender Unemployment Gap in Denmark (2007-2022)', x=0.5, font=dict(size=20)),
    xaxis=dict(title='', range=[2006.5, 2022.5], showticklabels=True, dtick=1),
    yaxis=dict(title='Percentage', range=[-6, 7], tickvals=[-6,-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]),
    height=700, width=1200,
    shapes=shapes,
    annotations=[
        dict(
            text='Remark: Data for GDP from 2019 to 2022 are preliminary data. Magenta lines indicate change in economy at 1.5%.',
            xref='paper', yref='paper', x=0, y=0.04, showarrow=False, font=dict(size=10)),
        dict(
            text='Source: Statistics Denmark AULP01 & NAN1',
            xref='paper', yref='paper', x=0, y=0.0, showarrow=False, font=dict(size=10))
    ]
)

# Create the figure and add the traces
fig_3 = go.Figure(data=[gdp_trace, gap_trace], layout=layout)

# Show the interactive plot
fig_3.show()

Figure 3 shows both Real GDP Growth and Gender Unemployment Gap plotted. There was a economical downturn in the periods 2007-2009, 2011-2013 and 2019-2020, and economic climate in the periods 2010, 2014-2018 and 2021-2022, which is illustrated by the vertical lines. In the first period of recession, Real GDP Growth is seen to fall significantly, where Gender Unemployment is also seen to fall sharply. This decrease in Gender Unemployment reflects higher unemployment among men than among women. The inequality in unemployment between men and women is seen to normalize, where there is also a higher Real GDP Growth - despite this being defined as a period of low economic activity. In the period after from 2014-2018, there is a boom, where Gender Unemployment is again a positive sign that unemployment is higher among women than for men. Although in the years 2016 and 2017 there is a significantly higher Real GDP Growth than in 2014 and 2018, the Gender Unemployment is stable during this period. It is also largely unaffected in the period 2019-2020, when COVID-19 takes place, where inequality falls slightly. In the last period 2021-2022, a strong percentage increase in Real GDP Growth is seen, where Gender Unemployment is also seen to rise, but not to such a high degree.

In addition to the visual illustration, we also want to mathematically calculate the correlation between the two variables - Real GDP Growth and Gender Unemployment Gap. This is done by first merging the GDP and Gender Gap dataframes on year, and then calculating the correlation using the function "corr".

For this result, the following applies:
- A positive correlation indicates that the two variables move in the same direction.
- A negative correlation indicates that the two variables move in opposite directions.
- A correlation close to zero indicates there is no strong relationship between the variables.

In [9]:
# Merge the GDP and gender_gap dataframes on 'YEAR'
merged_data = pd.merge(GDP, gender_gap, on='YEAR')

# Calculate the correlation between 'REAL GROWTH IN GDP (%)' and 'GENDER UNEMPLOYMENT GAP (%)'
correlation_fig3 = merged_data['REAL GROWTH IN GDP (%)'].corr(merged_data['GENDER UNEMPLOYMENT GAP (%)'])

# Print the correlation
print("Correlation between Real GDP Growth and Gender Unemployment Gap:", correlation_fig3)

Correlation between Real GDP Growth and Gender Unemployment Gap: 0.3599194870134143


We find that the correlation between them is 0.36, which suggests that there is some correlation between them, but that GDP is not the only explanatory factor for this inequality in Gender Unemployment.

We now want to investigate including a new factor - industry - in order to see if there should be an overrepresentation of unemployment in certain industries compared to others. It is used to calculate volatility for each industry, so that it can be ascertained which industries are extra exposed, and whether there should be a correlation between how exposed one's industry is and how high the Gender Unemployment Gap is. For this, data from table RAS300 is used, which comes from the DST API and shows the number of "Employed (end November) by industry (DB07), socioeconomic status, age and sex". A table is made which shows a summary of the data, where the parameters included are then defined. A fetching based on parameter set is performed and grouped based on industry, year and gender, which are added up so that the specific number can be found. The data is sorted and an extra column is added with the percentage change in total employment from year to year. The standard deviation of the 'CHANGE (%)' column for each industry and gender is then calculated and the columns for men and women are separated. Finally, there is the overall gender gap in volatility, which can now be illustrated as an interactive scatter plot using Plotly, seen as Figure 4. As before, the correlation between the overall gender gap and men's volatility for each industry is also calculated using the "corr" function.

In [10]:
# Searching for GDP data through DST API
employment_dst = DstApi('RAS300')

# Creating and displaying the a summary of the GDP data
summary_employment = employment_dst.tablesummary(language='en')
summary_employment

Table RAS300: Employed (end November) by industry (DB07), socioeconomic status, age, sex and time
Last update: 2022-11-16T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,BRANCHE07,140,TOT,"TOT Industry, total",99999,99999 Activity not stated,False
1,SOCIO,8,05,Self-employed,40,"Employees, not specified",False
2,ALDER,13,-15,Under 16 years,67-,67 years and over,False
3,KØN,3,TOT,Total,K,Women,False
4,Tid,14,2008,2008,2021,2021,True


In [11]:
# Setting parameters
par_employment = {
    'table': 'RAS300',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'BRANCHE07', 'values': ['1','2','3','4','5','6','7','8','9','10']}, # Choosing all relevant industries
        {'code': 'SOCIO', 'values': ['*']}, # Cannot leave empty so we choose all values with *
        {'code': 'ALDER', 'values': ['*']}, # Cannot leave empty so we choose all values with *
        {'code': 'KØN', 'values': ['TOT', 'M', 'K']}, # TOT = Total, M = Male, K = Female
        {'code': 'Tid', 'values': ['2006<=2021']}
    ]
}

# Fetching data based on parameters set
employment = employment_dst.get_data(params=par_employment)

# Grouping by industry, year, and gender, and summing the NUMBER OF EMPLOYEES column
employment = employment.groupby(['BRANCHE07', 'TID', 'KØN'], as_index=False)['INDHOLD'].sum()

# Remove the numbers at the beginning of the industry names
employment['BRANCHE07'] = employment['BRANCHE07'].str.replace(r'^\d+\s+', '', regex=True)

# Renaming the columns
employment.rename(columns=
                {'BRANCHE07':'INDUSTRY',
                'TID':'YEAR',
                'KØN':'GENDER',
                'INDHOLD':'NUMBER OF EMPLOYEES'},
                inplace=True
            )

# Sorting the DataFrame by industry, year, and gender
employment.sort_values(['INDUSTRY', 'YEAR', 'GENDER'], inplace=True)

# Adding a column with the percentage change in total employment from year to year
employment['CHANGE (%)'] = employment.groupby(['INDUSTRY', 'GENDER'])['NUMBER OF EMPLOYEES'].pct_change() * 100

# Drop rows where YEAR equals 2008
employment = employment[employment['YEAR'] != 2008]

# Displaying employment table
employment.head(6)

,INDUSTRY,YEAR,GENDER,NUMBER OF EMPLOYEES,CHANGE (%)
3,"Agriculture, forestry and fishing",2009,Men,59129,-2.157761
4,"Agriculture, forestry and fishing",2009,Total,74414,-3.077745
5,"Agriculture, forestry and fishing",2009,Women,15285,-6.479442
6,"Agriculture, forestry and fishing",2010,Men,56889,-3.788327
7,"Agriculture, forestry and fishing",2010,Total,71090,-4.466901
8,"Agriculture, forestry and fishing",2010,Women,14201,-7.091920


In [12]:
# Calculate the standard deviation of the 'CHANGE (%)' column for each industry and gender
volatility = employment.groupby(['INDUSTRY', 'GENDER'])['CHANGE (%)'].std().reset_index()

# Pivot the volatility table to have separate columns for male and female volatility
volatility_pivot = volatility.pivot_table(values='CHANGE (%)', index=['INDUSTRY'], columns=['GENDER']).reset_index()

# Calculate the overall gender gap in volatility
volatility_pivot['GENDER GAP'] = volatility_pivot['Women'] - volatility_pivot['Men']

# Create an interactive scatter plot using Plotly
fig_4 = px.scatter(volatility_pivot, x='Men', y='GENDER GAP', hover_name='INDUSTRY', title='Figure 4: Overall Gender Gap vs. Employment Volatility for Each Industry')

# Update the layout
fig_4.update_layout(xaxis_title='Volatility of Employment (%)', yaxis_title='Overall Gender Gap (%)', title_font=dict(size=20), 
                    title_x=0.5, margin=dict(l=50, r=50, b=50, t=50, pad=4), hovermode='closest',
                    height=500, width=1000,
                    annotations=[dict(xref='paper', yref='paper', x=0, y=-0, showarrow=False, text='Source: Statistics Denmark RAS300')])

# Show the interactive plot
fig_4.show()

In [13]:
correlation_fig4 = volatility_pivot['Men'].corr(volatility_pivot['GENDER GAP'])

# Print the correlation
print('The correlation between overall Gender Gap and employment volatility for each industry is', correlation_fig4)

The correlation between overall Gender Gap and employment volatility for each industry is -0.1857816614027788


It can be seen that women are fundamentally overrepresented in the unemployment statistics for most industries, as the Overall Gender Gap for most industries is positive, with Public administration, education and health being the clearest exceptions, where men have a higher unemployment rate than women. This industry is also characterized as being a strongly female-dominated profession and high job security in particularly low economic times, as it is state-run, which could be the best explanatory factor for the industry's position on the figure. As for the other industries, a downward trend is seen with a calculated correlation of approximately -0.18, which means that there is a correlation between the more exposed an industry is to unemployment, the lower the overall Gender Gap is also seen to be. If we look at the types of occupations that are most at risk, it is seen to be primarily a male-dominated industry, such as Construction, Manufacturing, mining and quarrying, and utility services. They are also the ones with some of the lower Gender Gap rates. There is some indication that in periods of recession there could be a tendency for industries that are very exposed and affected by the economic cycle to also have a higher proportion of men, which could be a causal explanation for some of the difference in unemployment that is seen between men and women.

---
## Overall Conclusion

In conclusion, we first found out that unemployment in Denmark has been exposed to two spikes in the current period from 2007-2022, which is due to the two crises - the financial crisis and COVID-19, which the economy has been exposed to during the period . In addition, a general inequality in unemployment was found far between men and women, with women seen to be overrepresented in the unemployment statistics for most of the period. However, the financial crisis is seen to be an exception, where the roles were reversed and men were the most exposed to unemployment. The reason for this shift was an interesting question to raise and was attempted to be answered by first looking at the economic situation and its connection with the Gender Unemployment Gap. Here a correlation was found, but was not enough to answer the question in detail. It therefore became interesting to look at the individual industries and see if there should be a correlation between how exposed they were to unemployment and how high a Gender Gap is seen for the individual industries. Here it was found that in the industries which are highly exposed and affected by the economic cycle, there is also a large over-representation of men, which could explain why we see the big change during the financial crisis. It is important to emphasize that this study cannot by any means conclude that as the only factors. Of other important explanatory factors for the development of inequality in unemployment between men and women. Alternatively, one could also examine differences within e.g. education, geography (countries), origin, age distribution and institutional explanatory factors. However, since this project tries to keep a simplicity for the big topic, these otherwise important factors are not explored.